# The Data Analysis Bureau Exercise

In [6]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.vector_ar.var_model import VAR
import xgboost as xgb

## Data import and inspection

In [2]:
# Open JSON file and assign to dict
f = open('/Users/sam/Downloads/DS 2021 Exercise/data_dict.json')
data_dict = json.load(f)
f.close()

In [3]:
# Print line-by-line to fit everything
for key in data_dict.keys():
    print(data_dict[key])

{'name': 'CurrentSpeed', 'units': 'knots', 'range': None, 'description': ' '}
{'name': 'CurrentDir', 'units': 'degrees', 'range': None, 'description': ' '}
{'name': 'TWS', 'units': 'knots', 'range': None, 'description': 'True Wind Speed'}
{'name': 'TWA', 'units': 'degrees', 'range': None, 'description': 'True Wind Angle'}
{'name': 'AWS', 'units': 'knots', 'range': None, 'description': 'Apparent Wind Speed'}
{'name': 'AWA', 'units': 'degrees', 'range': None, 'description': 'Apparent Wind Angle'}
{'name': 'Roll', 'units': 'degrees', 'range': None, 'description': 'Roll, also equals to -Heel'}
{'name': 'Pitch', 'units': 'degrees', 'range': None, 'description': 'Pitch angle'}
{'name': 'HeadingMag', 'units': 'degrees', 'range': None, 'description': 'magnetic heading'}
{'name': 'HoG', 'units': 'degrees', 'range': None, 'description': 'heading over ground'}
{'name': 'HeadingTrue', 'units': 'degrees', 'range': None, 'description': 'true heading. True heading - heading over ground = Yaw'}
{'name

In [16]:
# Import data and inspect top
data = pd.read_csv('/Users/sam/Downloads/test_data.csv')
data.head()

,CurrentSpeed,CurrentDir,TWS,TWA,AWS,AWA,Roll,Pitch,HeadingMag,HoG,...,VMG,RudderAng,Leeway,TWD,WSoG,VoltageDrawn,ModePilote,DateTime,Yaw,Tacking
0,0.0756,123.0,10.8,48.0,10.4,48.0,-3.54,9.080000,24.0,308.0,...,0.0594,4.666667,0.0,356.0,10.5,11.8,5.0,2019-04-14 00:00:00.000,-299.0,1.0
1,0.0756,123.0,10.8,48.0,10.4,48.0,-3.54,9.080000,24.0,308.0,...,0.0594,4.666667,0.0,356.0,10.5,11.8,5.0,2019-04-14 00:00:01.000,-299.0,1.0
2,0.0756,123.0,10.8,48.0,10.4,48.0,-3.52,9.099999,24.0,308.0,...,0.0594,4.666667,0.0,356.0,9.9,11.8,5.0,2019-04-14 00:00:02.000,-299.0,1.0
3,0.0756,123.0,10.8,48.0,10.4,48.0,-3.52,9.099999,24.0,308.0,...,0.0594,4.666667,0.0,356.0,9.9,11.8,5.0,2019-04-14 00:00:03.000,-299.0,1.0
4,0.0756,123.0,10.8,48.0,10.4,48.0,-3.50,9.099999,24.0,308.0,...,0.0594,4.666667,0.0,356.0,10.3,11.8,5.0,2019-04-14 00:00:04.000,-299.0,1.0


In [17]:
# Convert date to DT
data['DateTime'] = pd.to_datetime(data['DateTime'])

In [19]:
# Check column types
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220000 entries, 0 to 219999
Data columns (total 27 columns):
CurrentSpeed    219833 non-null float64
CurrentDir      219832 non-null float64
TWS             219837 non-null float64
TWA             219833 non-null float64
AWS             219834 non-null float64
AWA             219838 non-null float64
Roll            219833 non-null float64
Pitch           219836 non-null float64
HeadingMag      219835 non-null float64
HoG             219838 non-null float64
HeadingTrue     219837 non-null float64
AirTemp         219840 non-null float64
Longitude       219836 non-null float64
Latitude        219840 non-null float64
SoG             219842 non-null float64
SoS             219840 non-null float64
AvgSoS          219838 non-null float64
VMG             219837 non-null float64
RudderAng       219838 non-null float64
Leeway          219839 non-null float64
TWD             219838 non-null float64
WSoG            219836 non-null float64
VoltageDr

In [20]:
# Describe numerical data
data.iloc[:,12:].describe()

,Longitude,Latitude,SoG,SoS,AvgSoS,VMG,RudderAng,Leeway,TWD,WSoG,VoltageDrawn,ModePilote,Yaw,Tacking
count,219836.000000,219840.000000,219842.000000,219840.000000,219838.000000,219837.000000,219838.000000,219839.000000,219838.000000,219836.000000,219839.000000,219839.000000,219834.000000,219995.000000
mean,-60.675999,16.805625,7.658572,7.607856,6.382535,4.109238,2.025093,-1.226548,83.323893,14.746686,12.417475,2.422614,5.595763,0.209273
std,0.982475,3.929849,3.075285,3.084592,2.903147,2.066419,4.963518,0.793367,53.810225,4.291340,0.570748,1.043669,140.521400,0.406791
min,-61.816873,11.971172,0.005400,0.000000,0.070200,0.000000,-37.333336,-10.000000,0.000000,0.000000,11.100000,2.000000,-359.000000,0.000000
25%,-61.639917,12.913855,7.684200,7.597800,5.637600,3.148200,-0.666667,-2.000000,61.000000,11.800000,12.100000,2.000000,-13.000000,0.000000
50%,-61.199546,15.232683,8.726400,8.699400,7.581600,4.579200,2.333333,-1.000000,70.000000,14.800000,12.300000,2.000000,-6.000000,0.000000
75%,-59.735390,20.902214,9.369000,9.298800,8.461801,5.583600,4.666667,-1.000000,82.000000,17.600000,12.500000,2.000000,-1.000000,0.000000
max,-59.279375,22.209945,12.598201,12.700800,8.532001,9.860400,47.000000,9.000000,359.000000,35.700001,14.200000,5.000000,359.000000,1.000000


In [22]:
for col in data.columns:
    print(f'{col}: {data[col].isna().sum()}')

CurrentSpeed: 0
CurrentDir: 0
TWS: 0
TWA: 0
AWS: 0
AWA: 0
Roll: 0
Pitch: 0
HeadingMag: 0
HoG: 0
HeadingTrue: 0
AirTemp: 0
Longitude: 0
Latitude: 0
SoG: 0
SoS: 0
AvgSoS: 0
VMG: 0
RudderAng: 0
Leeway: 0
TWD: 0
WSoG: 0
VoltageDrawn: 0
ModePilote: 0
DateTime: 0
Yaw: 0
Tacking: 0


## Cleaning data

Interpolate NaN in DT as we know it samples once per second. So if DT is NaN at index 1, we can insert the value as DT at index 0 plus one second.

In [19]:
# Get index of missing in DT
inds = pd.isnull(data['DateTime']).nonzero()[0]
inds

/Users/sam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  


array([ 39959,  81738,  82751,  91468, 121548])

In [20]:
# For each missing value, we add the previous timestamp plus 1
# Note that we run chronologically down DF
# Thus, if there are multiple NaN in a row (not the case), this would be fine
for item in inds:
    data.loc[item, 'DateTime'] = data.loc[item-1, 'DateTime'] + pd.to_timedelta(1, unit='s')

Other missing values could be interpolated, as we might assume that things such as direction and speed might not change drastically from one time point to another. This could be done by taking an average of the n previous and following values and setting it in place of a missing value. This would exclude the Tacking and ModePilote columns as they seem categorical.

In [21]:
# Interpolating for all other numerical variables
# Input mean of previous 5 and following 5 values
for col in [item for item in data.columns if item not in ['ModePilote', 'Tacking']]:
    inds = pd.isnull(data[col]).to_numpy().nonzero()[0]
    for i in inds:
           data.loc[i, col] = data.loc[(i-5):(i+5), col].mean()

# For categorical variables, we take only the previous value
# Trade-off between labels being correct and complete dataset
for col in ['ModePilote', 'Tacking']:
    inds = pd.isnull(data[col]).to_numpy().nonzero()[0]
    for i in inds:
           data.loc[i, col] = data.loc[i-1, col].mean()

In [23]:
# Scale numeric features
num_cols = [item for item in data.columns if item not in ['ModePilote', 'Tacking', 'DateTime']]
X_numeric = data[num_cols]
scaled_features = StandardScaler().fit_transform(X_numeric.values)
scaled_df = pd.DataFrame(scaled_features, columns=X_numeric.columns)

In [24]:
# Check if data is stationary

# Augmented Dickey-Fuller Test (ADF Test)/unit root test
def adf_test(ts, signif=0.05):
    dftest = adfuller(ts, autolag='AIC')
    adf = pd.Series(dftest[0:4], index=['Test Statistic','p-value','# Lags','# Observations'])
    for key,value in dftest[4].items():
       adf['Critical Value (%s)'%key] = value
    #print(adf)
    
    p = adf['p-value']
    if p <= signif:
        print(f" Series is Stationary")
    else:
        print(f" Series is Non-Stationary")

# Look at all numerical features
for col in scaled_df.columns:
    print(col)
    adf_test(scaled_df[col])

CurrentSpeed
Test Statistic         -8.072558e+00
p-value                 1.535612e-12
# Lags                  8.300000e+01
# Observations          2.199160e+05
Critical Value (1%)    -3.430380e+00
Critical Value (5%)    -2.861553e+00
Critical Value (10%)   -2.566777e+00
dtype: float64
 Series is Stationary
CurrentDir
Test Statistic         -1.336628e+01
p-value                 5.305422e-25
# Lags                  8.200000e+01
# Observations          2.199170e+05
Critical Value (1%)    -3.430380e+00
Critical Value (5%)    -2.861553e+00
Critical Value (10%)   -2.566777e+00
dtype: float64
 Series is Stationary
TWS
Test Statistic         -1.117620e+01
p-value                 2.567195e-20
# Lags                  8.300000e+01
# Observations          2.199160e+05
Critical Value (1%)    -3.430380e+00
Critical Value (5%)    -2.861553e+00
Critical Value (10%)   -2.566777e+00
dtype: float64
 Series is Stationary
TWA
Test Statistic         -1.370807e+01
p-value                 1.252792e-25
# Lags

In [ ]:
# Choose forecasting horizon (e.g. 15-30s)
forecasting_horizon = 15

# This is only if we use all data to predict final bit
# Look into rolling prediction and evaluation
train = data.head(-forecasting_horizon).copy()
y_valid = data.tail(forecasting_horizon)['y']

# Include lags in dataset (could be more/less/different)
train['lag-15'] = train['y'].shift(15)
train['lag-30'] = train['y'].shift(30)

# drop NAs
train_without_nulls = train.dropna()
X_train = train_without_nulls.drop(columns='y')
y_train = train_without_nulls['y']

## Modeling: VAR, XGBoost, LSTM

In [ ]:
# VAR model
model = VAR(endog=train)

In [ ]:
# XGBoost

# Walk-forward evaluation